<a href="https://colab.research.google.com/github/Geuens/curso_ai/blob/main/Copy_of_persistent_preguntas_y_respuestas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preguntas y Respuestas con RAG Langchain

Usando Chroma DB and LangChain para contestar preguntas de un texto, con una base de datos local. Guardamos los embaddings, y los usamos después.

In [ ]:
!pip install chromadb langchain langchain-openai

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings

In [ ]:
from google.colab import userdata
openai_token = userdata.get('OPENAI_TOKEN')


## Cargar y procesar documentos.
aqui cargamos los documentos sobre los que vamosa  aplicar el Q&A

Los dividimos en trozos (chunks). ASi buscamos los más relevantes y los pasamos al LLM.

In [ ]:
# Load and process the text
loader = TextLoader('state_of_the_union.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

## Inicializamos PeristedChromaDB

Creamos los embeddings. Y los guardamos en la db

In [ ]:

# Directorio donde se guardará la db
persist_directory = 'db'

engine = "gpt-4"
embedding = OpenAIEmbeddings(api_key=openai_token, model="text-embedding-3-large")

vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)

## Persist the Database
In a notebook, we should call `persist()` to ensure the embeddings are written to disk.
This isn't necessary in a script - the database will be automatically persisted when the client object is destroyed.

In [ ]:
vectordb.persist()
vectordb = None

## Cargamos la db del disco
`persist_directory` es el directorio donde se va a guardar la db y `embedding_function` tiene que ser la misma que cunado instanciamos a db. Se encargara de las respuestas.

In [ ]:
# Cargamos la base de datos persistente del disco y usamos un langchain retrieval
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(api_key=openai_token),
    retriever=vectordb.as_retriever(
        search_kwargs={"k": 5})
    )

## Preguntas y respuestas

Con langchainy RAG

In [ ]:
query = "Cuál es el sentido de la vida"
result = qa.invoke(query)
result.get("result")

' Lo siento, pero no puedo responder esa pregunta ya que soy un programa de computadora diseñado para proporcionar información basada en datos y contexto específico. La pregunta sobre el sentido de la vida es más adecuada para una discusión filosófica o personal.'

## Limpieza

Cuando hayas terminado con la base de datos, puedes eliminarla del disco. Puedes eliminar la colección específica con la que está trabajando (si tiene varias) o eliminar toda la base de datos destruyendo el directorio de persistencia.

In [ ]:
# limpiamos la colección
vectordb.delete_collection()
vectordb.persist()

# nos cargamos el directorio de persistencia
!rm -rf db/

Persisting DB to disk, putting it in the save folder db
